# Joe's Code

In [1]:
# Imports
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
""" Ticker Library init
portfolio = {'Sector':[],'Company Name':[],'Ticker':[]}
hello = [
    ['Technology','Netflix','NFLX'],
    ['Technology','Meta','FB'],
    ['Technology','Uber','UBER'],
    ['Technology','Microchip Technology','MCHP'],
    ['Technology','AirBnB','ABNB'],
    ['Energy','Diamondback Energy Inc.','FANG'],
    ['Energy','Marathon Oil Corp.','MRO'],
    ['Energy','Devon Energy Corp.','DVN'],
    ['Energy','SunPower Corp.','SPWR'],
    ['Energy','Renewable Energy Group Inc.','REGI'],
    ['Finance','McKinsey & Company','MTRX'],
    ['Finance','BlackRock','BLK'],
    ['Finance','PayPal','PYPL'],
    ['Finance','Mercadolibre Inc','MELI'],
    ['Finance','SoFi','SOFI']
]
for row in hello:
    portfolio['Sector'].append(row[0])
    portfolio['Company Name'].append(row[1])
    portfolio['Ticker'].append(row[2])
portfolio = pd.DataFrame(portfolio).set_index('Sector')
portfolio.to_csv('./Data/Cleaned_Data/Ticker_library.csv')
"""

" Ticker Library init\nportfolio = {'Sector':[],'Company Name':[],'Ticker':[]}\nhello = [\n    ['Technology','Netflix','NFLX'],\n    ['Technology','Meta','FB'],\n    ['Technology','Uber','UBER'],\n    ['Technology','Microchip Technology','MCHP'],\n    ['Technology','AirBnB','ABNB'],\n    ['Energy','Diamondback Energy Inc.','FANG'],\n    ['Energy','Marathon Oil Corp.','MRO'],\n    ['Energy','Devon Energy Corp.','DVN'],\n    ['Energy','SunPower Corp.','SPWR'],\n    ['Energy','Renewable Energy Group Inc.','REGI'],\n    ['Finance','McKinsey & Company','MTRX'],\n    ['Finance','BlackRock','BLK'],\n    ['Finance','PayPal','PYPL'],\n    ['Finance','Mercadolibre Inc','MELI'],\n    ['Finance','SoFi','SOFI']\n]\nfor row in hello:\n    portfolio['Sector'].append(row[0])\n    portfolio['Company Name'].append(row[1])\n    portfolio['Ticker'].append(row[2])\nportfolio = pd.DataFrame(portfolio).set_index('Sector')\nportfolio.to_csv('./Data/Cleaned_Data/Ticker_library.csv')\n"

## Python Edgar (Joe's Code)

*LK Comment*
### **Section 1**
#### Uses SEC Edgar Downloader to download all 10-Q (quarterly reports) since June 1, 2012 (10 years) for a defined set of companies from the SEC Edgar database.

In [16]:
tickers = pd.read_csv('./Data/Cleaned_Data/Ticker_library.csv')['Ticker'].to_list()

In [17]:
from sec_edgar_downloader import Downloader

In [18]:
def create_SEC(path,tickers,filing_type,after_date,before_date=None):
    dl = Downloader(f'{path}')
    for ticker in tickers:
        dl.get(filing_type, ticker, after=after_date,before=before_date)

In [19]:
# Uncomment to download
# create_SEC('./Data/Raw_Data/',tickers,"10-Q",after_date='2012-06-01')

*LK Comment*
### **Section 2**
#### Extract information from the SEC filings and construct an organized dataframe.

In [20]:
def txt_finder():
    txts = []
    for root, dirs, files in os.walk(".", topdown=False):
        for name in files:
            os.path.join(root, name)
            if name[-4:]=='.txt':
                txts.append(f'{root}/{name}')
        for name in dirs:
            os.path.join(root, name)
    return txts
sec_paths = txt_finder()

In [34]:
print(sec_paths)

['./Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/0001193125-12-462895/full-submission.txt', './Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/0000866273-20-000135/full-submission.txt', './Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/0000866273-18-000025/full-submission.txt', './Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/0000866273-17-000018/full-submission.txt', './Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/0000866273-18-000128/full-submission.txt', './Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/0000866273-15-000094/full-submission.txt', './Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/0000866273-18-000008/full-submission.txt', './Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/0000866273-17-000122/full-submission.txt', './Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/0000866273-14-000006/full-submission.txt', './Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/0000866273-22-000008/full-submission.txt', './Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/0000866273-14-000055/full-submission.txt', './Data/Raw_Data/sec-edgar-fili

In [35]:
def path_part(path,before,after):
    beforeNum = path.find(before)+len(before)
    afterNum = path.find(after)
    return path[beforeNum:afterNum]

def sec_df_from_paths(sec_paths,filing_type):
    data = {
        'Ticker':[],
        'DocName':[],
        'DocPath':[],
        'FilingType':[],
        'DocText':[]
    }
    for sec in sec_paths:
        data['Ticker'].append(path_part(sec,'sec-edgar-filings/',f'/{filing_type}'))
        data['DocName'].append(path_part(sec,f'/{filing_type}/','/full-submission'))
        data['DocPath'].append(sec)
        data['FilingType'].append(filing_type)
        sec_txt = open(sec, "r")
        data['DocText'].append(sec_txt.read())
        sec_txt.close()
    return pd.DataFrame(data)

df_10Q = sec_df_from_paths(sec_paths,'10-Q')

In [36]:
df_10Q.head().tail(2)

,Ticker,DocName,DocPath,FilingType,DocText
3,MTRX,0000866273-17-000018,./Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/00...,10-Q,<SEC-DOCUMENT>0000866273-17-000018.txt : 20170...
4,MTRX,0000866273-18-000128,./Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/00...,10-Q,<SEC-DOCUMENT>0000866273-18-000128.txt : 20181...


In [23]:
import re
def restore_windows_1252_characters(restore_string):
    """
        Replace C1 control characters in the Unicode string s by the
        characters at the corresponding code points in Windows-1252,
        where possible.
    """

    def to_windows_1252(match):
        try:
            return bytes([ord(match.group(0))]).decode('windows-1252')
        except UnicodeDecodeError:
            # No character at the corresponding code point: remove it.
            return ''
        
    return re.sub(r'[\u0080-\u0099]', to_windows_1252, restore_string)

In [24]:
# df_10Q['DocText'].apply(restore_windows_1252_characters)
df_10Q_test = df_10Q.head()
df_10Q_test['DocText'].apply(restore_windows_1252_characters)

0    <SEC-DOCUMENT>0001193125-12-462895.txt : 20121...
1    <SEC-DOCUMENT>0000866273-20-000135.txt : 20201...
2    <SEC-DOCUMENT>0000866273-18-000025.txt : 20180...
3    <SEC-DOCUMENT>0000866273-17-000018.txt : 20170...
4    <SEC-DOCUMENT>0000866273-18-000128.txt : 20181...
Name: DocText, dtype: object

In [25]:
df_10Q['DocText'].loc[0][:1000]

'<SEC-DOCUMENT>0001193125-12-462895.txt : 20121109\n<SEC-HEADER>0001193125-12-462895.hdr.sgml : 20121109\n<ACCEPTANCE-DATETIME>20121109143534\nACCESSION NUMBER:\t\t0001193125-12-462895\nCONFORMED SUBMISSION TYPE:\t10-Q\nPUBLIC DOCUMENT COUNT:\t\t12\nCONFORMED PERIOD OF REPORT:\t20120930\nFILED AS OF DATE:\t\t20121109\nDATE AS OF CHANGE:\t\t20121109\n\nFILER:\n\n\tCOMPANY DATA:\t\n\t\tCOMPANY CONFORMED NAME:\t\t\tMATRIX SERVICE CO\n\t\tCENTRAL INDEX KEY:\t\t\t0000866273\n\t\tSTANDARD INDUSTRIAL CLASSIFICATION:\tCONSTRUCTION SPECIAL TRADE CONTRACTORS [1700]\n\t\tIRS NUMBER:\t\t\t\t731352174\n\t\tSTATE OF INCORPORATION:\t\t\tDE\n\t\tFISCAL YEAR END:\t\t\t0630\n\n\tFILING VALUES:\n\t\tFORM TYPE:\t\t10-Q\n\t\tSEC ACT:\t\t1934 Act\n\t\tSEC FILE NUMBER:\t001-15461\n\t\tFILM NUMBER:\t\t121193087\n\n\tBUSINESS ADDRESS:\t\n\t\tSTREET 1:\t\t5100 EAST SKELLY DRIVE\n\t\tSTREET 2:\t\tSUITE 700\n\t\tCITY:\t\t\tTULSA\n\t\tSTATE:\t\t\tOK\n\t\tZIP:\t\t\t74135\n\t\tBUSINESS PHONE:\t\t9188388822\n\n\tMAIL

In [ ]:
from sec_api import ExtractorApi
sec_api = os.getenv("sec_api_key")
extractorApi = ExtractorApi("sec_api_key")

# Tesla 10-K filing
filing_url = "https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/tsla-10k_20201231.htm"

# get the standardized and cleaned text of section 1A "Risk Factors"
#section_text = extractorApi.get_section(filing_url, "1A", "text")

#print(section_text)

In [30]:
len("CONFORMED PERIOD OF REPORT:\t")

28

In [31]:
def DocText_extraction(text,before,after):
    return text[
        text.find(before)+len(before):
        text.find(after)
        ]

In [32]:
def SEC_date(DocText):
    """Returns Conformed Period of Report when given the text of an SEC report"""
    return DocText_extraction(DocText,"CONFORMED PERIOD OF REPORT:\t","\nFILED AS OF DATE:")
def SEC_CIK(DocText):
    """Returns CENTRAL INDEX KEY when given the text of an SEC report"""
    return DocText_extraction(DocText,"CENTRAL INDEX KEY:\t\t\t","\n\t\tSTANDARD INDUSTRIAL CLASSIFICATION:")
    

In [38]:
def extract_all_SEC_dates(df:pd.DataFrame):
    df['SECdates'] = df['DocText'].apply(SEC_date)
def extract_all_SEC_CIK(df:pd.DataFrame):
    df['SEC_CIK'] = df['DocText'].apply(SEC_CIK)
extract_all_SEC_dates(df_10Q)
extract_all_SEC_CIK(df_10Q)
df_10Q.head()

,Ticker,DocName,DocPath,FilingType,DocText,SECdates,SEC_CIK
0,MTRX,0001193125-12-462895,./Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/00...,10-Q,<SEC-DOCUMENT>0001193125-12-462895.txt : 20121...,20120930,0000866273
1,MTRX,0000866273-20-000135,./Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/00...,10-Q,<SEC-DOCUMENT>0000866273-20-000135.txt : 20201...,20200930,0000866273
2,MTRX,0000866273-18-000025,./Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/00...,10-Q,<SEC-DOCUMENT>0000866273-18-000025.txt : 20180...,20180331,0000866273
3,MTRX,0000866273-17-000018,./Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/00...,10-Q,<SEC-DOCUMENT>0000866273-17-000018.txt : 20170...,20170331,0000866273
4,MTRX,0000866273-18-000128,./Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/00...,10-Q,<SEC-DOCUMENT>0000866273-18-000128.txt : 20181...,20180930,0000866273


In [39]:
df_10Q.to_csv('sec.csv')

In [40]:
df_10Q.tail()

,Ticker,DocName,DocPath,FilingType,DocText,SECdates,SEC_CIK
422,DVN,0001564590-20-021639,./Data/Raw_Data/sec-edgar-filings/DVN/10-Q/000...,10-Q,<SEC-DOCUMENT>0001564590-20-021639.txt : 20200...,20200331,0001090012
423,DVN,0001564590-18-010177,./Data/Raw_Data/sec-edgar-filings/DVN/10-Q/000...,10-Q,<SEC-DOCUMENT>0001564590-18-010177.txt : 20180...,20180331,0001090012
424,DVN,0001193125-14-397861,./Data/Raw_Data/sec-edgar-filings/DVN/10-Q/000...,10-Q,<SEC-DOCUMENT>0001193125-14-397861.txt : 20141...,20140930,0001090012
425,DVN,0001564590-20-049203,./Data/Raw_Data/sec-edgar-filings/DVN/10-Q/000...,10-Q,<SEC-DOCUMENT>0001564590-20-049203.txt : 20201...,20200930,0001090012
426,DVN,0001564590-20-036568,./Data/Raw_Data/sec-edgar-filings/DVN/10-Q/000...,10-Q,<SEC-DOCUMENT>0001564590-20-036568.txt : 20200...,20200630,0001090012



### **LK START HERE** 
#### First creating a text df for the SEC data

In [23]:
import os
import pandas as pd
import re
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# CSV with text is very slow - create a df that removes the text column. 
# All code commented out because created new "No Text" csv
# cols_list = ["Ticker", "DocName", "DocPath", "FilingType", "SECdates", "SEC_CIK"]
# df_10Q_notext = pd.read_csv('../Notebooks/sec.csv', usecols=cols_list)
# df_10Q_notext.to_csv("sec_notext.csv")

In [7]:
df_10Q_notext = pd.read_csv('../Notebooks/sec_notext.csv')
df_10Q_notext.head()

,Unnamed: 0,Ticker,DocName,DocPath,FilingType,SECdates,SEC_CIK
0,0,MTRX,0001193125-12-462895,./Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/00...,10-Q,20120930,866273
1,1,MTRX,0000866273-20-000135,./Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/00...,10-Q,20200930,866273
2,2,MTRX,0000866273-18-000025,./Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/00...,10-Q,20180331,866273
3,3,MTRX,0000866273-17-000018,./Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/00...,10-Q,20170331,866273
4,4,MTRX,0000866273-18-000128,./Data/Raw_Data/sec-edgar-filings/MTRX/10-Q/00...,10-Q,20180930,866273


In [12]:
df_10Q_notext = df_10Q_notext.drop(columns='Unnamed: 0')


In [17]:
df_10Q_notext = df_10Q_notext.sort_values(by=['Ticker','SECdates'])

In [203]:
df_10Q_notext.head()

,Ticker,DocName,DocPath,FilingType,SECdates,SEC_CIK,urls,url_V1,url_V2
367,ABNB,0001628280-21-010389,./Data/Raw_Data/sec-edgar-filings/ABNB/10-Q/00...,10-q,20210331,1559720,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/000155...
368,ABNB,0001628280-21-016979,./Data/Raw_Data/sec-edgar-filings/ABNB/10-Q/00...,10-q,20210630,1559720,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/000155...
369,ABNB,0001559720-21-000017,./Data/Raw_Data/sec-edgar-filings/ABNB/10-Q/00...,10-q,20210930,1559720,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/000155...
366,ABNB,0001559720-22-000009,./Data/Raw_Data/sec-edgar-filings/ABNB/10-Q/00...,10-q,20220331,1559720,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/000155...
69,BLK,0001193125-12-343728,./Data/Raw_Data/sec-edgar-filings/BLK/10-Q/000...,10-q,20120630,1364742,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/000136...


### Attempting to assemble URLs via Frankenstein method

In [57]:
def url_maker_10q(df:pd.DataFrame):
    #V1 Example:
    # "https://www.sec.gov/ix?doc=/Archives/edgar/data/1364742/000156459022018505/blk-10q_20220331.htm"
    # url_base:                 https://www.sec.gov/ix?doc=/Archives/edgar/data/
    # SEC CIK:                  1364742
    # SEC DOCName:              000156459022018505
    # Ticker:                   blk
    # Doc Type:                 10q
    # Date (date 2022-03-31):   20220331

    #V2 Example
    # "https://www.sec.gov/Archives/edgar/data/0001090012/000156459018018317/dvn-10q_20180630.htm"

    url_base = 'https://www.sec.gov/ix?doc=/Archives/edgar/data/'
    url_base2 = 'https://www.sec.gov/Archives/edgar/data/000'
    SEC_cik = df['SEC_CIK'].apply(str)
    SEC_DocName = df['DocName'].apply(lambda x: re.sub('-','',x))
    ticker = df['Ticker'].apply(str.lower)
    df['FilingType'] = df['FilingType'].apply(str.lower)
    doc_type = df['FilingType'].apply(lambda x: re.sub('-','',x))
    date = df['SECdates'].apply(str)
    df['url_V1'] = url_base + SEC_cik + '/' + SEC_DocName + '/' + ticker + '-' + doc_type + '_' + date + '.htm'
    df['url_V2'] = url_base2 + SEC_cik + '/' + SEC_DocName + '/' + ticker + '-' + doc_type + '_' + date + '.htm'
    return df

In [58]:
urls = url_maker_10q(df_10Q_notext)


In [62]:
df_10Q_notext.head()

,Ticker,DocName,DocPath,FilingType,SECdates,SEC_CIK,urls,url_V1,url_V2
367,ABNB,0001628280-21-010389,./Data/Raw_Data/sec-edgar-filings/ABNB/10-Q/00...,10-q,20210331,1559720,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/000155...
368,ABNB,0001628280-21-016979,./Data/Raw_Data/sec-edgar-filings/ABNB/10-Q/00...,10-q,20210630,1559720,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/000155...
369,ABNB,0001559720-21-000017,./Data/Raw_Data/sec-edgar-filings/ABNB/10-Q/00...,10-q,20210930,1559720,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/000155...
366,ABNB,0001559720-22-000009,./Data/Raw_Data/sec-edgar-filings/ABNB/10-Q/00...,10-q,20220331,1559720,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/000155...
69,BLK,0001193125-12-343728,./Data/Raw_Data/sec-edgar-filings/BLK/10-Q/000...,10-q,20120630,1364742,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/000136...


In [60]:
df_10Q_notext.at[411, 'url_V2']

'https://www.sec.gov/Archives/edgar/data/0001090012/000156459018018317/dvn-10q_20180630.htm'

https://www.sec.gov/Archives/edgar/data/0001090012/000156459018018317/dvn-10q_20180630.htm

https://www.sec.gov/ix?doc=/Archives/edgar/data/1559720/000155972022000009/abnb-20220331.htm

### Returning to SEC API
https://sec-api.io/docs/sec-filings-item-extraction-api-python-example

In [61]:
from sec_api import QueryApi

queryApi = QueryApi(api_key="43d18e4ffedde4ca940e46ce210780daa0b7de17674abb369b5a8ef9fb3b69a1")

query = {
  "query": { "query_string": { 
      "query": "ticker:ABNB AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-Q\"" 
    } },
  "from": "0",
  "size": "10",
  "sort": [{ "filedAt": { "order": "desc" } }]
}

filings = queryApi.get_filings(query)

print(filings)

{'total': {'value': 4, 'relation': 'eq'}, 'query': {'from': 0, 'size': 10}, 'filings': [{'id': 'aad08488621ae941aa2b71563d4976d6', 'accessionNo': '0001559720-22-000009', 'cik': '1559720', 'ticker': 'ABNB', 'companyName': 'Airbnb, Inc.', 'companyNameLong': 'Airbnb, Inc. (Filer)', 'formType': '10-Q', 'description': 'Form 10-Q - Quarterly report [Sections 13 or 15(d)]', 'filedAt': '2022-05-09T16:48:00-04:00', 'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1559720/000155972022000009/0001559720-22-000009.txt', 'linkToHtml': 'https://www.sec.gov/Archives/edgar/data/1559720/000155972022000009/0001559720-22-000009-index.htm', 'linkToXbrl': '', 'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/1559720/000155972022000009/abnb-20220331.htm', 'entities': [{'companyName': 'Airbnb, Inc. (Filer)', 'cik': '1559720', 'irsNo': '263051428', 'stateOfIncorporation': 'DE', 'fiscalYearEnd': '1231', 'type': '10-Q', 'act': '34', 'fileNo': '001-39778', 'filmNo': '22905806', 'sic': '7340 Ser

In [65]:
df_abnb = pd.json_normalize(filings, record_path =['filings'])

In [67]:
df_abnb_trim = df_abnb[['ticker', 'companyName', 'cik', 'accessionNo', 'formType', 'periodOfReport', 'linkToFilingDetails']]
df_abnb_trim.head()

,ticker,companyName,cik,accessionNo,formType,periodOfReport,linkToFilingDetails
0,ABNB,"Airbnb, Inc.",1559720,0001559720-22-000009,10-Q,2022-03-31,https://www.sec.gov/Archives/edgar/data/155972...
1,ABNB,"Airbnb, Inc.",1559720,0001559720-21-000017,10-Q,2021-09-30,https://www.sec.gov/Archives/edgar/data/155972...
2,ABNB,"Airbnb, Inc.",1559720,0001628280-21-016979,10-Q,2021-06-30,https://www.sec.gov/Archives/edgar/data/155972...
3,ABNB,"Airbnb, Inc.",1559720,0001628280-21-010389,10-Q,2021-03-31,https://www.sec.gov/Archives/edgar/data/155972...


In [80]:
df_abnb_trim.at[0, 'linkToFilingDetails']

'https://www.sec.gov/Archives/edgar/data/1559720/000155972022000009/abnb-20220331.htm'

In [2]:
# Failed attempt to loop through the queries

from sec_api import QueryApi
queryApi = QueryApi(api_key="43d18e4ffedde4ca940e46ce210780daa0b7de17674abb369b5a8ef9fb3b69a1")
ticker_list = pd.read_csv("./Data/Cleaned_Data/Ticker_library.csv")["Ticker"].to_list()
file_df = {}
for ticker in ticker_list:
    query = {
        "query": { "query_string": { 
            "query": f"ticker:{ticker} AND filedAt:"+"{2012-06-01 TO 2022-06-01} AND formType:\"10-Q\"" 
        } },
        "from": "0",
        "size": "10",
        "sort": [{ "filedAt": { "order": "desc" } }]
    }
    filings = queryApi.get_filings(query)
    file_df[ticker] = pd.json_normalize(filings, record_path =['filings'])

file_df["BLK"]

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
0,142323fc045219e27243382f17e6eedb,0001564590-22-018505,1364742,BLK,BlackRock Inc.,BlackRock Inc. (Filer),10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2022-05-06T12:40:30-04:00,https://www.sec.gov/Archives/edgar/data/136474...,https://www.sec.gov/Archives/edgar/data/136474...,,https://www.sec.gov/Archives/edgar/data/136474...,"[{'companyName': 'BlackRock Inc. (Filer)', 'ci...","[{'sequence': '1', 'description': '10-Q', 'doc...","[{'sequence': '7', 'description': 'XBRL TAXONO...",[],2022-03-31,NaN
1,e34b499ce62ae2e586ceb76ef4c3672d,0001564590-21-054838,1364742,BLK,BlackRock Inc.,BlackRock Inc. (Filer),10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2021-11-05T13:55:20-04:00,https://www.sec.gov/Archives/edgar/data/136474...,https://www.sec.gov/Archives/edgar/data/136474...,,https://www.sec.gov/Archives/edgar/data/136474...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '10215157', 'docume...","[{'sequence': '6', 'size': '107698', 'document...",[],2021-09-30,2021-09-30
2,05efb9271cff317f8dd86f91442e13fd,0001564590-21-041406,1364742,BLK,BlackRock Inc.,BlackRock Inc. (Filer),10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2021-08-05T13:36:35-04:00,https://www.sec.gov/Archives/edgar/data/136474...,https://www.sec.gov/Archives/edgar/data/136474...,,https://www.sec.gov/Archives/edgar/data/136474...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '10152269', 'docume...","[{'sequence': '6', 'size': '108634', 'document...",[],2021-06-30,2021-06-30
3,41673df3c869682574ba8db8eb10fc57,0001564590-21-024726,1364742,BLK,BlackRock Inc.,BlackRock Inc. (Filer),10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2021-05-06T12:59:42-04:00,https://www.sec.gov/Archives/edgar/data/136474...,https://www.sec.gov/Archives/edgar/data/136474...,,https://www.sec.gov/Archives/edgar/data/136474...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '7192603', 'documen...","[{'sequence': '6', 'size': '108323', 'document...",[],2021-03-31,2021-03-31
4,e33564d4b98244ef8e00e9e81a14c0cc,0001564590-20-051202,1364742,BLK,BlackRock Inc.,BlackRock Inc. (Filer),10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2020-11-05T14:59:31-05:00,https://www.sec.gov/Archives/edgar/data/136474...,https://www.sec.gov/Archives/edgar/data/136474...,,https://www.sec.gov/Archives/edgar/data/136474...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '9993343', 'documen...","[{'sequence': '6', 'size': '110513', 'document...",[],2020-09-30,NaN
5,bf4dd6498200875d95b27f192586bbc1,0001564590-20-038433,1364742,BLK,BlackRock Inc.,BlackRock Inc. (Filer),10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2020-08-07T19:42:49-04:00,https://www.sec.gov/Archives/edgar/data/136474...,https://www.sec.gov/Archives/edgar/data/136474...,,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,"[{'companyName': 'BlackRock Inc. (Filer)', 'ci...","[{'sequence': '1', 'description': '10-Q', 'doc...","[{'sequence': '5', 'description': 'XBRL TAXONO...",[],2020-06-30,NaN
6,8221be9133ef102024439a4fb4c7901d,0001564590-20-023352,1364742,BLK,BlackRock Inc.,BlackRock Inc. (Filer),10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2020-05-08T12:02:08-04:00,https://www.sec.gov/Archives/edgar/data/136474...,https://www.sec.gov/Archives/edgar/data/136474...,,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,"[{'companyName': 'BlackRock Inc. (Filer)', 'ci...","[{'sequence': '1', 'description': '10-Q', 'doc...","[{'sequence': '6', 'description': 'XBRL TAXONO...",[],2020-03-31,NaN
7,73fee4df0ebe6f7f29b4624076313652,0001564590-19-042157,1364742,BLK,BlackRock Inc.,BlackRock Inc. (Filer),10-Q,Form 10-Q - Quarterly report [Sections 

## Query Mania

In [120]:
blk_query = {
    "query": { "query_string": { 
        "query": "ticker:BLK AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-Q\"" 
    } },
    "from": "0",
    "size": "50",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
blk_filings = queryApi.get_filings(blk_query)
blk_df = pd.json_normalize(blk_filings, record_path =['filings'])
blk_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
29,1b438522858c3bba8192661b29c27170,0001193125-12-343728,1364742,BLK,BlackRock Inc.,BlackRock Inc. (Filer),10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2012-08-08T15:00:59-04:00,https://www.sec.gov/Archives/edgar/data/136474...,https://www.sec.gov/Archives/edgar/data/136474...,,https://www.sec.gov/Archives/edgar/data/136474...,"[{'fiscalYearEnd': '0226', 'stateOfIncorporati...","[{'sequence': '1', 'size': '2638050', 'documen...","[{'sequence': '7', 'size': '3429637', 'documen...",[],NaN,NaN


In [119]:
dvn_query = {
    "query": { "query_string": { 
        "query": "ticker:DVN AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-Q\"" 
    } },
    "from": "0",
    "size": "50",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
dvn_filings = queryApi.get_filings(dvn_query)
dvn_df = pd.json_normalize(dvn_filings, record_path =['filings'])
dvn_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
29,fa9ccad5449a80c6d2f4e563e8149f6c,0001193125-12-329113,1090012,DVN,DEVON ENERGY CORP/DE,DEVON ENERGY CORP/DE (Filer),10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2012-08-01T17:07:07-04:00,https://www.sec.gov/Archives/edgar/data/109001...,https://www.sec.gov/Archives/edgar/data/109001...,,https://www.sec.gov/Archives/edgar/data/109001...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '1071464', 'documen...","[{'sequence': '6', 'size': '3049336', 'documen...",[],NaN,NaN


In [118]:
fang_query = {
    "query": { "query_string": { 
        "query": "ticker:FANG AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-Q\"" 
    } },
    "from": "0",
    "size": "50",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
fang_filings = queryApi.get_filings(fang_query)
fang_df = pd.json_normalize(fang_filings, record_path =['filings'])
fang_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
31,adf4b9112bc7c0442245731a0926ccb3,0001539838-12-000004,1539838,FANG,"Diamondback Energy, Inc.","Diamondback Energy, Inc. (Filer)",10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2012-11-15T20:17:19-05:00,https://www.sec.gov/Archives/edgar/data/153983...,https://www.sec.gov/Archives/edgar/data/153983...,,https://www.sec.gov/Archives/edgar/data/153983...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '956131', 'document...",[],[],NaN,NaN


In [125]:
fb_query = {
    "query": { "query_string": { 
        "query": "ticker:FB AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-Q\"" 
    } },
    "from": "0",
    "size": "60",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
fb_filings = queryApi.get_filings(fb_query)
fb_df = pd.json_normalize(fb_filings, record_path =['filings'])
fb_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
29,47c6fca753e8d0a7095973f32c69ad99,0001193125-12-325997,1326801,FB,Facebook Inc,Facebook Inc (Filer),10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2012-07-31T16:39:19-04:00,https://www.sec.gov/Archives/edgar/data/132680...,https://www.sec.gov/Archives/edgar/data/132680...,,https://www.sec.gov/Archives/edgar/data/132680...,"[{'act': '34', 'cik': '0001326801', 'fileNo': ...","[{'sequence': '1', 'size': '830550', 'document...","[{'sequence': '12', 'size': '851211', 'documen...",[],NaN,NaN


In [126]:
JPM_query = {
    "query": { "query_string": { 
        "query": "ticker:JPM AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-Q\"" 
    } },
    "from": "0",
    "size": "60",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
JPM_filings = queryApi.get_filings(JPM_query)
JPM_df = pd.json_normalize(JPM_filings, record_path =['filings'])
JPM_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
30,52617c77a9ce7cf0b0934a36481055c1,0000019617-12-000262,19617,JPM,JPMORGAN CHASE & CO,JPMORGAN CHASE & CO (Filer),10-Q/A,Form 10-Q/A - Quarterly report [Sections 13 or...,2012-08-09T07:44:21-04:00,https://www.sec.gov/Archives/edgar/data/19617/...,https://www.sec.gov/Archives/edgar/data/19617/...,,https://www.sec.gov/Archives/edgar/data/19617/...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '10603182', 'docume...","[{'sequence': '8', 'size': '17768089', 'docume...",[],NaN,NaN


In [127]:
MCHP_query = {
    "query": { "query_string": { 
        "query": "ticker:MCHP AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-Q\"" 
    } },
    "from": "0",
    "size": "60",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
MCHP_filings = queryApi.get_filings(MCHP_query)
MCHP_df = pd.json_normalize(MCHP_filings, record_path =['filings'])
MCHP_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
29,a8742740e951b6baf39e355fd55f1ed9,0000827054-12-000230,827054,MCHP,MICROCHIP TECHNOLOGY INC,MICROCHIP TECHNOLOGY INC (Filer),10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2012-08-09T14:11:12-04:00,https://www.sec.gov/Archives/edgar/data/827054...,https://www.sec.gov/Archives/edgar/data/827054...,,https://www.sec.gov/Archives/edgar/data/827054...,"[{'fiscalYearEnd': '0331', 'stateOfIncorporati...","[{'sequence': '1', 'size': '995432', 'document...","[{'sequence': '5', 'size': '1132614', 'documen...",[],NaN,NaN


In [128]:
MELI_query = {
    "query": { "query_string": { 
        "query": "ticker:MELI AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-Q\"" 
    } },
    "from": "0",
    "size": "60",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
MELI_filings = queryApi.get_filings(MELI_query)
MELI_df = pd.json_normalize(MELI_filings, record_path =['filings'])
MELI_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
30,57c84389e7f92c435c2df9dc49a6ea81,0001193125-12-334988,1099590,MELI,MERCADOLIBRE INC,MERCADOLIBRE INC (Filer),10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2012-08-03T16:42:06-04:00,https://www.sec.gov/Archives/edgar/data/109959...,https://www.sec.gov/Archives/edgar/data/109959...,,https://www.sec.gov/Archives/edgar/data/109959...,"[{'act': '34', 'cik': '0001099590', 'fileNo': ...","[{'sequence': '1', 'size': '1163079', 'documen...","[{'sequence': '7', 'size': '1276563', 'documen...",[],NaN,NaN


In [129]:
MRO_query = {
    "query": { "query_string": { 
        "query": "ticker:MRO AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-Q\"" 
    } },
    "from": "0",
    "size": "60",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
MRO_filings = queryApi.get_filings(MRO_query)
MRO_df = pd.json_normalize(MRO_filings, record_path =['filings'])
MRO_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
30,95ad481b92e358678eaaf4c6c42879e3,0001140361-12-035104,101778,MRO,MARATHON OIL CORP,MARATHON OIL CORP (Filer),10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2012-08-03T17:02:59-04:00,https://www.sec.gov/Archives/edgar/data/101778...,https://www.sec.gov/Archives/edgar/data/101778...,,https://www.sec.gov/Archives/edgar/data/101778...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '1805365', 'documen...","[{'sequence': '7', 'size': '1129144', 'documen...",[],NaN,NaN


In [130]:
MTRX_query = {
    "query": { "query_string": { 
        "query": "ticker:MTRX AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-Q\"" 
    } },
    "from": "0",
    "size": "60",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
MTRX_filings = queryApi.get_filings(MTRX_query)
MTRX_df = pd.json_normalize(MTRX_filings, record_path =['filings'])
MTRX_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
29,163a2acd5ecc9c696e4b3367c24c631a,0001193125-12-462895,866273,MTRX,MATRIX SERVICE CO,MATRIX SERVICE CO (Filer),10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2012-11-09T14:35:34-05:00,https://www.sec.gov/Archives/edgar/data/866273...,https://www.sec.gov/Archives/edgar/data/866273...,,https://www.sec.gov/Archives/edgar/data/866273...,"[{'fiscalYearEnd': '0630', 'stateOfIncorporati...","[{'sequence': '1', 'size': '460125', 'document...","[{'sequence': '7', 'size': '510748', 'document...",[],NaN,NaN


In [131]:
NFLX_query = {
    "query": { "query_string": { 
        "query": "ticker:NFLX AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-Q\"" 
    } },
    "from": "0",
    "size": "60",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
NFLX_filings = queryApi.get_filings(NFLX_query)
NFLX_df = pd.json_normalize(NFLX_filings, record_path =['filings'])
NFLX_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
30,fa8e8b4f6df16eeaa0d96045c1ac6581,0001065280-12-000012,1065280,NFLX,NETFLIX INC,NETFLIX INC (Filer),10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2012-08-01T19:27:12-04:00,https://www.sec.gov/Archives/edgar/data/106528...,https://www.sec.gov/Archives/edgar/data/106528...,,https://www.sec.gov/Archives/edgar/data/106528...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '1477526', 'documen...","[{'sequence': '5', 'size': '1330995', 'documen...",[],NaN,NaN


In [132]:
PYPL_query = {
    "query": { "query_string": { 
        "query": "ticker:PYPL AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-Q\"" 
    } },
    "from": "0",
    "size": "60",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
PYPL_filings = queryApi.get_filings(PYPL_query)
PYPL_df = pd.json_normalize(PYPL_filings, record_path =['filings'])
PYPL_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
20,058de0a2b24121a58409d37af3a3f2c5,0001633917-15-000008,1633917,PYPL,"PayPal Holdings, Inc.","PayPal Holdings, Inc. (Filer)",10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2015-07-29T16:44:33-04:00,https://www.sec.gov/Archives/edgar/data/163391...,https://www.sec.gov/Archives/edgar/data/163391...,,https://www.sec.gov/Archives/edgar/data/163391...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '1623941', 'documen...","[{'sequence': '6', 'size': '1911158', 'documen...",[],NaN,NaN


In [133]:
REGI_query = {
    "query": { "query_string": { 
        "query": "ticker:REGI AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-Q\"" 
    } },
    "from": "0",
    "size": "60",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
REGI_filings = queryApi.get_filings(REGI_query)
REGI_df = pd.json_normalize(REGI_filings, record_path =['filings'])
REGI_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
29,4726a45582957d7de3ff9b5238b6aa6f,0001193125-12-355910,1463258,REGI,"Renewable Energy Group, Inc.","Renewable Energy Group, Inc. (Filer)",10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2012-08-14T17:11:10-04:00,https://www.sec.gov/Archives/edgar/data/146325...,https://www.sec.gov/Archives/edgar/data/146325...,,https://www.sec.gov/Archives/edgar/data/146325...,"[{'fiscalYearEnd': '1231', 'act': '34', 'cik':...","[{'sequence': '1', 'size': '854663', 'document...","[{'sequence': '6', 'size': '1186851', 'documen...",[],NaN,NaN


In [134]:
SPWR_query = {
    "query": { "query_string": { 
        "query": "ticker:SPWR AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-Q\"" 
    } },
    "from": "0",
    "size": "60",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
SPWR_filings = queryApi.get_filings(SPWR_query)
SPWR_df = pd.json_normalize(SPWR_filings, record_path =['filings'])
SPWR_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
30,76e2609402e0afe5615c6929db7a6963,0000867773-12-000044,867773,SPWR,SUNPOWER CORP,SUNPOWER CORP (Filer),10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2012-08-09T17:04:30-04:00,https://www.sec.gov/Archives/edgar/data/867773...,https://www.sec.gov/Archives/edgar/data/867773...,,https://www.sec.gov/Archives/edgar/data/867773...,"[{'fiscalYearEnd': '1230', 'stateOfIncorporati...","[{'sequence': '1', 'size': '2022994', 'documen...","[{'sequence': '6', 'size': '3184035', 'documen...",[],NaN,NaN


In [135]:
UBER_query = {
    "query": { "query_string": { 
        "query": "ticker:UBER AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-Q\"" 
    } },
    "from": "0",
    "size": "60",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
UBER_filings = queryApi.get_filings(UBER_query)
UBER_df = pd.json_normalize(UBER_filings, record_path =['filings'])
UBER_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
9,24ebb4ff49f180b808aa8ea79a243ecc,0001628280-19-007524,1543151,UBER,"Uber Technologies, Inc","Uber Technologies, Inc (Filer)",10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2019-06-03T20:32:11-04:00,https://www.sec.gov/Archives/edgar/data/154315...,https://www.sec.gov/Archives/edgar/data/154315...,,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,"[{'companyName': 'Uber Technologies, Inc (File...","[{'sequence': '1', 'description': '10-Q', 'doc...","[{'sequence': '5', 'description': 'XBRL TAXONO...",[],2019-03-31,NaN


In [140]:
ABNB_query = {
    "query": { "query_string": { 
        "query": "ticker:ABNB AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-Q\"" 
    } },
    "from": "0",
    "size": "60",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
ABNB_filings = queryApi.get_filings(ABNB_query)
ABNB_df = pd.json_normalize(ABNB_filings, record_path =['filings'])
ABNB_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
3,a1d411ffd162add5ec5a245d49d200e6,0001628280-21-010389,1559720,ABNB,"Airbnb, Inc.","Airbnb, Inc. (Filer)",10-Q,Form 10-Q - Quarterly report [Sections 13 or 1...,2021-05-14T17:18:14-04:00,https://www.sec.gov/Archives/edgar/data/155972...,https://www.sec.gov/Archives/edgar/data/155972...,,https://www.sec.gov/Archives/edgar/data/155972...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '1774209', 'documen...","[{'sequence': '10', 'size': '56019', 'document...",[],2021-03-31,2021-03-31


In [136]:
ABNB_10k_query = {
    "query": { "query_string": { 
        "query": "ticker:ABNB AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-K\"" 
    } },
    "from": "0",
    "size": "20",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
ABNB_10k_filings = queryApi.get_filings(ABNB_10k_query)
ABNB_10k_df = pd.json_normalize(ABNB_10k_filings, record_path =['filings'])
ABNB_10k_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
1,895c8fd79836f1635f5fec30004a11e2,0001559720-21-000010,1559720,ABNB,"Airbnb, Inc.","Airbnb, Inc. (Filer)",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2021-02-26T16:19:13-05:00,https://www.sec.gov/Archives/edgar/data/155972...,https://www.sec.gov/Archives/edgar/data/155972...,,https://www.sec.gov/Archives/edgar/data/155972...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '2521279', 'documen...",[],[],2020-12-31,2020-12-31


In [141]:
FANG_10k_query = {
    "query": { "query_string": { 
        "query": "ticker:FANG AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-K\"" 
    } },
    "from": "0",
    "size": "20",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
FANG_10k_filings = queryApi.get_filings(FANG_10k_query)
FANG_10k_df = pd.json_normalize(FANG_10k_filings, record_path =['filings'])
FANG_10k_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
11,f5e15ef740428ce5a6411042c7845948,0001539838-13-000004,1539838,FANG,"Diamondback Energy, Inc.","Diamondback Energy, Inc. (Filer)",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2013-03-01T16:52:42-05:00,https://www.sec.gov/Archives/edgar/data/153983...,https://www.sec.gov/Archives/edgar/data/153983...,,https://www.sec.gov/Archives/edgar/data/153983...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '2589122', 'documen...","[{'sequence': '9', 'size': '2310543', 'documen...",[],2012-12-31,NaN


In [142]:
BLK_10k_query = {
    "query": { "query_string": { 
        "query": "ticker:BLK AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-K\"" 
    } },
    "from": "0",
    "size": "20",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
BLK_10k_filings = queryApi.get_filings(BLK_10k_query)
BLK_10k_df = pd.json_normalize(BLK_10k_filings, record_path =['filings'])
BLK_10k_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
9,68046a912a42c850823db61a5e948600,0001193125-13-084957,1364742,BLK,BlackRock Inc.,BlackRock Inc. (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2013-02-28T17:48:24-05:00,https://www.sec.gov/Archives/edgar/data/136474...,https://www.sec.gov/Archives/edgar/data/136474...,,https://www.sec.gov/Archives/edgar/data/136474...,"[{'fiscalYearEnd': '0226', 'stateOfIncorporati...","[{'sequence': '1', 'size': '3426024', 'documen...","[{'sequence': '9', 'size': '4155004', 'documen...",[],2012-12-31,NaN


In [115]:
dvn_10k_query = {
    "query": { "query_string": { 
        "query": "ticker:DVN AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-K\"" 
    } },
    "from": "0",
    "size": "20",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
dvn_10k_filings = queryApi.get_filings(dvn_10k_query)
dvn_10k_df = pd.json_normalize(dvn_10k_filings, record_path =['filings'])
dvn_10k_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
9,45d6f36821c7efceb7f216ee66a51425,0001193125-13-068817,1090012,DVN,DEVON ENERGY CORP/DE,DEVON ENERGY CORP/DE (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2013-02-21T12:19:56-05:00,https://www.sec.gov/Archives/edgar/data/109001...,https://www.sec.gov/Archives/edgar/data/109001...,,https://www.sec.gov/Archives/edgar/data/109001...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '2869622', 'documen...","[{'sequence': '17', 'size': '4068677', 'docume...",[],2012-12-31,NaN


In [143]:
FB_10k_query = {
    "query": { "query_string": { 
        "query": "ticker:FB AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-K\"" 
    } },
    "from": "0",
    "size": "20",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
FB_10k_filings = queryApi.get_filings(FB_10k_query)
FB_10k_df = pd.json_normalize(FB_10k_filings, record_path =['filings'])
FB_10k_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
11,71e9064cb8d0ba0cdf1a44321d40d175,0001326801-13-000003,1326801,FB,Facebook Inc,Facebook Inc (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2013-02-01T16:43:01-05:00,https://www.sec.gov/Archives/edgar/data/132680...,https://www.sec.gov/Archives/edgar/data/132680...,,https://www.sec.gov/Archives/edgar/data/132680...,"[{'act': '34', 'cik': '1326801', 'fileNo': '00...","[{'sequence': '1', 'size': '2559059', 'documen...","[{'sequence': '11', 'size': '2179597', 'docume...",[],2012-12-31,NaN


In [144]:
JPM_10k_query = {
    "query": { "query_string": { 
        "query": "ticker:JPM AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-K\"" 
    } },
    "from": "0",
    "size": "20",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
JPM_10k_filings = queryApi.get_filings(JPM_10k_query)
JPM_10k_df = pd.json_normalize(JPM_10k_filings, record_path =['filings'])
JPM_10k_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
9,ba213dcc438a9170fb57ba6338bc5f02,0000019617-13-000221,19617,JPM,JPMORGAN CHASE & CO,JPMORGAN CHASE & CO (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2013-02-28T16:06:59-05:00,https://www.sec.gov/Archives/edgar/data/19617/...,https://www.sec.gov/Archives/edgar/data/19617/...,,https://www.sec.gov/Archives/edgar/data/19617/...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '16568129', 'docume...","[{'sequence': '10', 'size': '26744140', 'docum...",[],2012-12-31,NaN


In [145]:
MCHP_10k_query = {
    "query": { "query_string": { 
        "query": "ticker:MCHP AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-K\"" 
    } },
    "from": "0",
    "size": "20",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
MCHP_10k_filings = queryApi.get_filings(MCHP_10k_query)
MCHP_10k_df = pd.json_normalize(MCHP_10k_filings, record_path =['filings'])
MCHP_10k_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
10,4c60e60ab6e8dead53f4e6ce24be902b,0000827054-13-000171,827054,MCHP,MICROCHIP TECHNOLOGY INC,MICROCHIP TECHNOLOGY INC (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2013-05-30T14:55:45-04:00,https://www.sec.gov/Archives/edgar/data/827054...,https://www.sec.gov/Archives/edgar/data/827054...,,https://www.sec.gov/Archives/edgar/data/827054...,"[{'fiscalYearEnd': '0331', 'stateOfIncorporati...","[{'sequence': '1', 'size': '2473354', 'documen...","[{'sequence': '8', 'size': '2806858', 'documen...",[],2013-03-31,NaN


In [146]:
MELI_10k_query = {
    "query": { "query_string": { 
        "query": "ticker:MELI AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-K\"" 
    } },
    "from": "0",
    "size": "20",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
MELI_10k_filings = queryApi.get_filings(MELI_10k_query)
MELI_10k_df = pd.json_normalize(MELI_10k_filings, record_path =['filings'])
MELI_10k_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
10,bf9c0f26b666e448b57a4dbfbaf13628,0001193125-13-081326,1099590,MELI,MERCADOLIBRE INC,MERCADOLIBRE INC (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2013-02-27T22:00:32-05:00,https://www.sec.gov/Archives/edgar/data/109959...,https://www.sec.gov/Archives/edgar/data/109959...,,https://www.sec.gov/Archives/edgar/data/109959...,"[{'act': '34', 'cik': '1099590', 'fileNo': '00...","[{'sequence': '1', 'size': '2353048', 'documen...","[{'sequence': '8', 'size': '2584183', 'documen...",[],2012-12-31,NaN


In [147]:
MRO_10k_query = {
    "query": { "query_string": { 
        "query": "ticker:MRO AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-K\"" 
    } },
    "from": "0",
    "size": "20",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
MRO_10k_filings = queryApi.get_filings(MRO_10k_query)
MRO_10k_df = pd.json_normalize(MRO_10k_filings, record_path =['filings'])
MRO_10k_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
9,b4d0e8fe7653adf7ecec26db12fdc509,0000101778-13-000006,101778,MRO,MARATHON OIL CORP,MARATHON OIL CORP (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2013-02-22T12:40:55-05:00,https://www.sec.gov/Archives/edgar/data/101778...,https://www.sec.gov/Archives/edgar/data/101778...,,https://www.sec.gov/Archives/edgar/data/101778...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '5700226', 'documen...","[{'sequence': '25', 'size': '5382844', 'docume...",[],2012-12-31,NaN


In [148]:
MTRX_10k_query = {
    "query": { "query_string": { 
        "query": "ticker:MTRX AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-K\"" 
    } },
    "from": "0",
    "size": "20",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
MTRX_10k_filings = queryApi.get_filings(MTRX_10k_query)
MTRX_10k_df = pd.json_normalize(MTRX_10k_filings, record_path =['filings'])
MTRX_10k_df.tail(1)

,ticker,formType,accessionNo,cik,companyNameLong,effectivenessDate,companyName,linkToFilingDetails,description,linkToTxt,filedAt,documentFormatFiles,periodOfReport,entities,id,seriesAndClassesContractsInformation,linkToHtml,linkToXbrl,dataFiles
10,MTRX,10-K,0001193125-12-382978,866273,MATRIX SERVICE CO (Filer),NaN,MATRIX SERVICE CO,https://www.sec.gov/Archives/edgar/data/866273...,Form 10-K - Annual report [Section 13 and 15(d...,https://www.sec.gov/Archives/edgar/data/866273...,2012-09-06T16:25:17-04:00,"[{'sequence': '1', 'size': '1237316', 'documen...",NaN,"[{'fiscalYearEnd': '0630', 'stateOfIncorporati...",b8c6251354c1e86f38786a2570a6b697,[],https://www.sec.gov/Archives/edgar/data/866273...,,"[{'sequence': '11', 'size': '1434044', 'docume..."


In [149]:
NFLX_10k_query = {
    "query": { "query_string": { 
        "query": "ticker:NFLX AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-K\"" 
    } },
    "from": "0",
    "size": "20",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
NFLX_10k_filings = queryApi.get_filings(NFLX_10k_query)
NFLX_10k_df = pd.json_normalize(NFLX_10k_filings, record_path =['filings'])
NFLX_10k_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
11,73c550cbef01d1fba6a44ab484dff5db,0001065280-13-000008,1065280,NFLX,NETFLIX INC,NETFLIX INC (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2013-01-31T21:43:16-05:00,https://www.sec.gov/Archives/edgar/data/106528...,https://www.sec.gov/Archives/edgar/data/106528...,,https://www.sec.gov/Archives/edgar/data/106528...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '1983014', 'documen...","[{'sequence': '8', 'size': '1674270', 'documen...",[],2012-12-31,NaN


In [150]:
PYPL_10k_query = {
    "query": { "query_string": { 
        "query": "ticker:PYPL AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-K\"" 
    } },
    "from": "0",
    "size": "20",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
PYPL_10k_filings = queryApi.get_filings(PYPL_10k_query)
PYPL_10k_df = pd.json_normalize(PYPL_10k_filings, record_path =['filings'])
PYPL_10k_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
6,cf557fa0603df0accd8467a6c2207f4f,0001633917-16-000113,1633917,PYPL,"PayPal Holdings, Inc.","PayPal Holdings, Inc. (Filer)",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2016-02-11T16:44:06-05:00,https://www.sec.gov/Archives/edgar/data/163391...,https://www.sec.gov/Archives/edgar/data/163391...,,https://www.sec.gov/Archives/edgar/data/163391...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '2316973', 'documen...","[{'sequence': '11', 'size': '2748911', 'docume...",[],NaN,NaN


In [151]:
REGI_10k_query = {
    "query": { "query_string": { 
        "query": "ticker:REGI AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-K\"" 
    } },
    "from": "0",
    "size": "20",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
REGI_10k_filings = queryApi.get_filings(REGI_10k_query)
REGI_10k_df = pd.json_normalize(REGI_10k_filings, record_path =['filings'])
REGI_10k_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
10,e39143a60d78adeb430a29c03f79edef,0001193125-13-098354,1463258,REGI,"Renewable Energy Group, Inc.","Renewable Energy Group, Inc. (Filer)",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2013-03-08T15:17:09-05:00,https://www.sec.gov/Archives/edgar/data/146325...,https://www.sec.gov/Archives/edgar/data/146325...,,https://www.sec.gov/Archives/edgar/data/146325...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '2175021', 'documen...","[{'sequence': '10', 'size': '2899791', 'docume...",[],2012-12-31,NaN


In [152]:
SPWR_10k_query = {
    "query": { "query_string": { 
        "query": "ticker:SPWR AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-K\"" 
    } },
    "from": "0",
    "size": "20",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
SPWR_10k_filings = queryApi.get_filings(SPWR_10k_query)
SPWR_10k_df = pd.json_normalize(SPWR_10k_filings, record_path =['filings'])
SPWR_10k_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
12,9fc6ce492c7464a4e6a1dc2c6cc8277c,0000867773-12-000037,867773,SPWR,SUNPOWER CORP,SUNPOWER CORP (Filer),10-K/A,Form 10-K/A - Annual report [Section 13 and 15...,2012-06-28T21:04:11-04:00,https://www.sec.gov/Archives/edgar/data/867773...,https://www.sec.gov/Archives/edgar/data/867773...,,https://www.sec.gov/Archives/edgar/data/867773...,"[{'fiscalYearEnd': '1230', 'stateOfIncorporati...","[{'sequence': '1', 'size': '87296', 'documentU...",[],[],NaN,NaN


In [161]:
UBER_10k_query = {
    "query": { "query_string": { 
        "query": "ticker:UBER AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-K\"" 
    } },
    "from": "0",
    "size": "20",
    "sort": [{ "filedAt": { "order": "desc" } }]
}
UBER_10k_filings = queryApi.get_filings(UBER_10k_query)
UBER_10k_df = pd.json_normalize(UBER_10k_filings, record_path =['filings'])
UBER_10k_df.tail(1)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
2,f17ed6366e380be7c276ea38ff6215c0,0001543151-20-000010,1543151,UBER,"Uber Technologies, Inc","Uber Technologies, Inc (Filer)",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2020-03-02T16:12:35-05:00,https://www.sec.gov/Archives/edgar/data/154315...,https://www.sec.gov/Archives/edgar/data/154315...,,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,"[{'companyName': 'Uber Technologies, Inc (File...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '13', 'description': 'XBRL TAXON...",[],2019-12-31,NaN


## Concatenate Dataframes from Queries

In [162]:
df_list = [ABNB_df, blk_df, dvn_df, fang_df, fb_df, JPM_df, MCHP_df, MELI_df, MRO_df, MTRX_df, NFLX_df, PYPL_df, REGI_df, SPWR_df, UBER_df, ABNB_10k_df, BLK_10k_df, dvn_10k_df, FANG_10k_df, FB_10k_df, JPM_10k_df, MCHP_10k_df, MELI_10k_df, MRO_10k_df, MTRX_10k_df, NFLX_10k_df, PYPL_10k_df, REGI_10k_df, SPWR_10k_df, UBER_10k_df]

In [163]:
df_concat = pd.concat(df_list, ignore_index=True)
df_concat.tail(5)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,linkToHtml,linkToXbrl,linkToFilingDetails,entities,documentFormatFiles,dataFiles,seriesAndClassesContractsInformation,periodOfReport,effectivenessDate
542,c6d3c5b7821235415c957a8751e25774,0000867773-13-000012,867773,SPWR,SUNPOWER CORP,SUNPOWER CORP (Filer),10-K,Form 10-K - Annual report [Section 13 and 15(d...,2013-02-22T19:37:51-05:00,https://www.sec.gov/Archives/edgar/data/867773...,https://www.sec.gov/Archives/edgar/data/867773...,,https://www.sec.gov/Archives/edgar/data/867773...,"[{'fiscalYearEnd': '1230', 'stateOfIncorporati...","[{'sequence': '1', 'size': '3838643', 'documen...","[{'sequence': '19', 'size': '5073559', 'docume...",[],2012-12-30,NaN
543,9fc6ce492c7464a4e6a1dc2c6cc8277c,0000867773-12-000037,867773,SPWR,SUNPOWER CORP,SUNPOWER CORP (Filer),10-K/A,Form 10-K/A - Annual report [Section 13 and 15...,2012-06-28T21:04:11-04:00,https://www.sec.gov/Archives/edgar/data/867773...,https://www.sec.gov/Archives/edgar/data/867773...,,https://www.sec.gov/Archives/edgar/data/867773...,"[{'fiscalYearEnd': '1230', 'stateOfIncorporati...","[{'sequence': '1', 'size': '87296', 'documentU...",[],[],NaN,NaN
544,aee6e093a9f6e6e1b6a6e1e36e334415,0001543151-22-000008,1543151,UBER,"Uber Technologies, Inc","Uber Technologies, Inc (Filer)",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2022-02-24T16:33:34-05:00,https://www.sec.gov/Archives/edgar/data/154315...,https://www.sec.gov/Archives/edgar/data/154315...,,https://www.sec.gov/Archives/edgar/data/154315...,"[{'companyName': 'Uber Technologies, Inc (File...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '9', 'description': 'XBRL TAXONO...",[],2021-12-31,NaN
545,559d64cb9f15ce7e161ba0aaad661fe0,0001543151-21-000014,1543151,UBER,"Uber Technologies, Inc","Uber Technologies, Inc (Filer)",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2021-03-01T08:06:48-05:00,https://www.sec.gov/Archives/edgar/data/154315...,https://www.sec.gov/Archives/edgar/data/154315...,,https://www.sec.gov/Archives/edgar/data/154315...,"[{'fiscalYearEnd': '1231', 'stateOfIncorporati...","[{'sequence': '1', 'size': '5326822', 'documen...","[{'sequence': '9', 'size': '149393', 'document...",[],2020-12-31,2020-12-31
546,f17ed6366e380be7c276ea38ff6215c0,0001543151-20-000010,1543151,UBER,"Uber Technologies, Inc","Uber Technologies, Inc (Filer)",10-K,Form 10-K - Annual report [Section 13 and 15(d...,2020-03-02T16:12:35-05:00,https://www.sec.gov/Archives/edgar/data/154315...,https://www.sec.gov/Archives/edgar/data/154315...,,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,"[{'companyName': 'Uber Technologies, Inc (File...","[{'sequence': '1', 'description': '10-K', 'doc...","[{'sequence': '13', 'description': 'XBRL TAXON...",[],2019-12-31,NaN


In [164]:
df_concat.to_csv('secURLs_fullinfo.csv')

In [165]:
url_df_trim = df_concat[['ticker', 'companyName', 'cik', 'accessionNo', 'formType', 'periodOfReport', 'linkToFilingDetails']]

In [166]:
url_df_trim.to_csv('secURLs_trim.csv')

In [167]:
list_10k_only = [ABNB_10k_df, BLK_10k_df, dvn_10k_df, FANG_10k_df, FB_10k_df, JPM_10k_df, MCHP_10k_df, MELI_10k_df, MRO_10k_df, MTRX_10k_df, NFLX_10k_df, PYPL_10k_df, REGI_10k_df, SPWR_10k_df, UBER_10k_df]
url_df_10k_only = pd.concat(list_10k_only, ignore_index=True)

ticker_list = ['ABNB', 'BLK', 'DVN', 'FANG', 'FB', 'JPM', 'MCHP', 'MELI', 'MRO', 'MTRX', 'NFLX', 'PYPL', 'REGI', 'SPWR', 'UBER']

## Testing Extractor API

In [159]:
url_df_trim.at[0, 'linkToFilingDetails']

'https://www.sec.gov/Archives/edgar/data/1559720/000155972022000009/abnb-20220331.htm'

In [202]:
from sec_api import ExtractorApi
sec_api = os.getenv("sec_api_key")
extractorApi = ExtractorApi(sec_api)

In [93]:
filing_url = df_abnb_trim['linkToFilingDetails'].astype(str)
print(filing_url)

0    https://www.sec.gov/Archives/edgar/data/155972...
1    https://www.sec.gov/Archives/edgar/data/155972...
2    https://www.sec.gov/Archives/edgar/data/155972...
3    https://www.sec.gov/Archives/edgar/data/155972...
Name: linkToFilingDetails, dtype: object


In [95]:
get_section_text(df_abnb_trim)
df_abnb_trim

InvalidSchema: No connection adapters were found for '0    https://api.sec-api.io/extractor?token=43d18e4...\n1    https://api.sec-api.io/extractor?token=43d18e4...\n2    https://api.sec-api.io/extractor?token=43d18e4...\n3    https://api.sec-api.io/extractor?token=43d18e4...\nName: linkToFilingDetails, dtype: object'

In [92]:
from sec_api import ExtractorApi
sec_api = os.getenv("sec_api_key")
extractorApi = ExtractorApi("43d18e4ffedde4ca940e46ce210780daa0b7de17674abb369b5a8ef9fb3b69a1")

# Tesla 10-K filing
filing_url = "https://www.sec.gov/Archives/edgar/data/1559720/000155972022000009/abnb-20220331.htm"

# get the standardized and cleaned text of section 1A "Risk Factors"
section_text = extractorApi.get_section(filing_url, "3", "text")

print(section_text)

undefined


In [201]:
filing_url = "https://www.sec.gov/Archives/edgar/data/789019/000156459021002316/msft-10q_20201231.htm"

# get the standardized and cleaned text of section 1A "Risk Factors"
section_text = extractorApi.get_section(filing_url, "part1item2", "text")

print(section_text)

 ITEM 2. MANAGEMENT&#8217;S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS

Note About Forward-Looking Statements

This report includes estimates, projections, statements relating to our business plans, objectives, and expected operating results that are &#8220;forward-looking statements&#8221; within the meaning of the Private Securities Litigation Reform Act of 1995, Section 27A of the Securities Act of 1933, and Section 21E of the Securities Exchange Act of 1934. Forward-looking statements may appear throughout this report, including the following sections: &#8220;Management&#8217;s Discussion and Analysis of Financial Condition and Results of Operations&#8221; and &#8220;Risk Factors&#8221; (Part II, Item 1A of this Form 10-Q). These forward-looking statements generally are identified by the words &#8220;believe,&#8221; &#8220;project,&#8221; &#8220;expect,&#8221; &#8220;anticipate,&#8221; &#8220;estimate,&#8221; &#8220;intend,&#8221; &#8220;strategy,&#822

## Behind the Pay Wall for Extractor API

### All 10-K and 10-Q sections can be extracted:

**10-K Sections**
* 1 - Business
* 1A - Risk Factors
* 1B - Unresolved Staff Comments
* 2 - Properties
* 3 - Legal Proceedings
* 4 - Mine Safety Disclosures
* 5 - Market for Registrant’s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities
* 6 - Selected Financial Data (prior to February 2021)
* 7 - Management’s Discussion and Analysis of Financial Condition and Results of Operations
* 7A - Quantitative and Qualitative Disclosures about Market Risk
* 8 - Financial Statements and Supplementary Data
* 9 - Changes in and Disagreements with Accountants on Accounting and Financial Disclosure
* 9A - Controls and Procedures
* 9B - Other Information
* 10 - Directors, Executive Officers and Corporate Governance
* 11 - Executive Compensation
* 12 - Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters
* 13 - Certain Relationships and Related Transactions, and Director Independence
* 14 - Principal Accountant Fees and Services

**10-Q Sections**

Part 1:
* 1 - Financial Statements
* 2 - Management’s Discussion and Analysis of Financial Condition and Results of Operations
* 3 - Quantitative and Qualitative Disclosures About Market Risk
* 4 - Controls and Procedures
  
Part 2:
* 1 - Legal Proceedings
* 1A - Risk Factors
* 2 -Unregistered Sales of Equity Securities and Use of Proceeds
* 3 - Defaults Upon Senior Securities
* 4 - Mine Safety Disclosures
* 5 - Other Information
* 6 - Exhibits
> https://pypi.org/project/sec-api/

### Request Parameters

url (required) - URL of the 10-K or 10-Q filing, e.g. TSLA 10-K https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/tsla-10k_20201231.htm.

item (required) - The item to be extracted, e.g. 1A. One item per API call is supported.

10-K supported item codes: 1, 1A, 1B, 2, 3, 4, 5, 6, 7, 7A, 8, 9, 9A, 9B, 10, 11, 12, 13, 14, 15

10-Q supported item codes: part1item1, part1item2, part1item3, part1item4, part2item1, part2item1a, part2item2, part2item3, part2item4, part2item5, part2item6

type (optional) - The return type of the item can be text or html. text returns clear, formatted text without any XBRL, XML or HTML tags. HTML entities such as &#160; are not removed. All tables are removed. The text return type is commonly used for NLP tasks. html returns the orignal, cleaned HTML version of the item including tables. Default: text

token (required) - Your API key.

### Request Examples
Get item 1A (Risk Factors) in clear text from Tesla's recent 10-K filing:

    https://api.sec-api.io/extractor?
    url=https://www.sec.gov/Archives/edgar/data/1318605/000156459021004599/tsla-10k_20201231.htm&
    item=1A&
    type=text&
    token=YOUR_API_KEY

Get item 8 (Financial Statements and Supplementary Data) as HTML from Apple's recent 10-K filing:

    https://api.sec-api.io/extractor?
    url=https://www.sec.gov/Archives/edgar/data/320193/000032019321000056/aapl-20210327.htm&
    item=8&
    type=html&
    token=YOUR_API_KEY

Get item 1A in part 2 (Risk Factors) as HTML from Microsoft's recent 10-Q filing:

    https://api.sec-api.io/extractor?
    url=https://www.sec.gov/Archives/edgar/data/789019/000156459022015675/msft-10q_20220331.htm&
    item=part2item1a&
    type=html&
    token=YOUR_API_KEY

### Creating a dataframe to cross-reference the different sections in 10-Q and 10-K SEC filings

In [5]:
sec_pull = {'sectionNo': ['1', '1A', '7', 'part1item2', 'part2item1A'],
            'sectionFullName': ['Business', 'Risk Factors', 'Management’s Discussion and Analysis of Financial Condition and Results of Operations', 'Management’s Discussion and Analysis of Financial Condition and Results of Operations', 'Risk Factors']}
sec_section_df = pd.DataFrame(sec_pull)
sec_section_df.to_csv('../Notebooks/Data/Cleaned_Data/sec_sections.csv')

In [172]:
data = {'sectionNo_10K': ['1', '1A', '1B', '2', '3', '4', '5', '6', '7', '7A', '8', '9', '9A', '9B', '10', '11', '12', '13', '14', 'na', 'na'],
        'sectionFullName_10k': ['Business', 'Risk Factors', 'Unresolved Staff Comments', 'Properties', 'Legal Proceedings', 'Mine Safety Disclosures', 'Market for Registrant’s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities', 'Selected Financial Data (prior to February 2021)', 'Management’s Discussion and Analysis of Financial Condition and Results of Operations', 'Quantitative and Qualitative Disclosures about Market Risk', 'Financial Statements and Supplementary Data', 'Changes in and Disagreements with Accountants on Accounting and Financial Disclosure', 'Controls and Procedures', 'Other Information', 'Directors, Executive Officers and Corporate Governance', 'Executive Compensation', 'Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters', 'Certain Relationships and Related Transactions, and Director Independence', 'Principal Accountant Fees and Services', 'na', 'na'],
        'sectionName_10k': ['1-Business', '1A-Risk Factors', '1B-Unresolved Staff Comments', '2-Properties', '3-Legal Proceedings', '4-Mine Safety', '5-Market for Equity Matters', '6-Select Fin Data pre-Feb21', '7-Mgmt Analysis', '7A-Market Risk', '8-Fin Statements', '9-Accounting Discrepancies', '9A-Controls', '9B-Other Info', '10-Corporate Governance', '11-Exec Comp', '12-Beneficial Owners', '13-Director Independence', '14-Accountant Fees', 'part2item3-NA', 'part2item6-NA'],
        'sectionName_10q': ['1-NA-10q', 'part2item1A-Risk Factors', '1B-NA-10q', '2-NA-10q', 'part2item1-Legal Proceedings', 'part2item4-Mine Safety', 'part2item2-Unreg Sales of Equity', '6-NA-10q', 'part1item2-Mgmt Analysis', 'part1item3-Market Risk', 'part1item1-Fin Statements', '9-NA-10q', 'part1item4-Control', 'part2item5-Other Info', '10-NA-10q', '11-NA-10q', '12-NA-10q', '13-NA-10q', '14-NA-10q', 'part2item3-Defaults on Securities', 'part2item6-Exh'],
        'sectionNo_10q': ['na', 'part2item1A', 'na', 'na', 'part2item1', 'part2item4', 'part2item2', 'na', 'part1item2', 'part1item3', 'part1item1', 'na', 'part1item4', 'part2item5', 'na', 'na', 'na', 'na', 'na', 'part2item3', 'part2item6'],
        'sectionFullName_10q': ['na', 'Risk Factors', 'na', 'na', 'Legal Proceedings', 'Mine Safety Disclosures', 'Unregistered Sales of Equity Securities and Use of Proceeds', 'na', 'Management’s Discussion and Analysis of Financial Condition and Results of Operations', 'Quantitative and Qualitative Disclosures About Market Risk', 'Financial Statements', 'na', 'Controls and Procedures', 'Other Information', 'na', 'na', 'na', 'na', 'na', 'Defaults Upon Senior Securities', 'Exhibits']}

sec_section_df = pd.DataFrame(data)
sec_section_df.head(20)

,sectionNo_10K,sectionFullName_10k,sectionName_10k,sectionName_10q,sectionNo_10q,sectionFullName_10q
0,1,Business,1-Business,1-NA-10q,na,na
1,1A,Risk Factors,1A-Risk Factors,part2item1A-Risk Factors,part2item1A,Risk Factors
2,1B,Unresolved Staff Comments,1B-Unresolved Staff Comments,1B-NA-10q,na,na
3,2,Properties,2-Properties,2-NA-10q,na,na
4,3,Legal Proceedings,3-Legal Proceedings,part2item1-Legal Proceedings,part2item1,Legal Proceedings
5,4,Mine Safety Disclosures,4-Mine Safety,part2item4-Mine Safety,part2item4,Mine Safety Disclosures
6,5,"Market for Registrant’s Common Equity, Related...",5-Market for Equity Matters,part2item2-Unreg Sales of Equity,part2item2,Unregistered Sales of Equity Securities and Us...
7,6,Selected Financial Data (prior to February 2021),6-Select Fin Data pre-Feb21,6-NA-10q,na,na
8,7,Management’s Discussion and Analysis of Financ...,7-Mgmt Analysis,part1item2-Mgmt Analysis,part1item2,Management’s Discussion and Analysis of Financ...
9,7A,Quantitative and Qualitative Disclosures about...,7A-Market Risk,part1item3-Market Risk,part1item3,Quantitative and Qualitative Disclosures About...


In [178]:
# SEC Sections as Lists
sectionNo_10k = ['1', '1A', '1B', '2', '3', '4', '5', '6', '7', '7A', '8', '9', '9A', '9B', '10', '11', '12', '13', '14']
sectionName_10k = ['1-Business', '1A-Risk Factors', '1B-Unresolved Staff Comments', '2-Properties', '3-Legal Proceedings', '4-Mine Safety', '5-Market for Equity Matters', '6-Select Fin Data pre-Feb21', '7-Mgmt Analysis', '7A-Market Risk', '8-Fin Statements', '9-Accounting Discrepancies', '9A-Controls', '9B-Other Info', '10-Corporate Governance', '11-Exec Comp', '12-Beneficial Owners', '13-Director Independence', '14-Accountant Fees']
list_10k_variables = [Business_1, RiskFactors_1A, UnresolvedStaffComments_1B, Properties_2, LegalProceedings_3, MineSafety_4, MarketforEquityMatters_5, SelectFinDatapreFeb21_6, MgmtAnalysis_7, MarketRisk_7A, FinStatements_8, AccountingDiscrepancies_9, Controls_9A, OtherInfo_9B, CorporateGovernance_10, ExecComp_11, BeneficialOwners_12, DirectorIndependence_13, AccountantFees_14]
sectionNo_10q = ['part1item1','part1item2','part1item3','part1item4','part2item1','part2item1A','part2item2','part2item3','part2item4','part2item5','part2item6']
sectionName_10q = ['part1item1-Fin Statements','part1item2-Mgmt Analysis','part1item3-Market Risk','part1item4-Control','part2item1-Legal Proceedings','part2item1A-Risk Factors','part2item2-Unreg Sales of Equity','part2item3-Defaults on Securities','part2item4-Mine Safety','part2item5-Other Info','part2item6-Exh']

### Testing Extractor API

In [192]:
extractorApi = ExtractorApi("43d18e4ffedde4ca940e46ce210780daa0b7de17674abb369b5a8ef9fb3b69a1")
filing_url = url_df_10k_only['linkToFilingDetails'].astype(str)
Business_1 = []
for url in filing_url:
    text = extractorApi.get_section(url, "1", "text")
    Business_1.append(text)


In [194]:
data = {"url": url_df_10k_only['linkToFilingDetails'].astype(str), "Business_1": Business_1}

In [195]:
Business_1_df = pd.DataFrame(data)
Business_1_df.tail()

,url,Business_1
140,https://www.sec.gov/Archives/edgar/data/867773...,ITEM 1: BUSINESS \n\nCorporate History \n\nWe...
141,https://www.sec.gov/Archives/edgar/data/867773...,undefined
142,https://www.sec.gov/Archives/edgar/data/154315...,ITEM 1. BUSINESS \n\nOverview \n\nUber Techno...
143,https://www.sec.gov/Archives/edgar/data/154315...,ITEM 1. BUSINESS \n\nOverview \n\nWe are a te...
144,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,ITEM 1. BUSINESS \n\nOverview \n\nWe are a te...


### Extractor API Function

In [196]:
# Function to create 10-k text dataframe

def create_10k_textdf():
    extractorApi = ExtractorApi("43d18e4ffedde4ca940e46ce210780daa0b7de17674abb369b5a8ef9fb3b69a1")
    filing_url = url_df_10k_only['linkToFilingDetails'].astype(str)
    Business_1 = [] 
    RiskFactors_1A = [] 
    UnresolvedStaffComments_1B = [] 
    Properties_2 = [] 
    LegalProceedings_3 = [] 
    MineSafety_4 = [] 
    MarketforEquityMatters_5 = [] 
    SelectFinDatapreFeb21_6 = [] 
    MgmtAnalysis_7 = [] 
    MarketRisk_7A = [] 
    FinStatements_8 = [] 
    AccountingDiscrepancies_9 = [] 
    Controls_9A = [] 
    OtherInfo_9B = [] 
    CorporateGovernance_10 = [] 
    ExecComp_11 = [] 
    BeneficialOwners_12 = [] 
    DirectorIndependence_13 = [] 
    AccountantFees_14 = []
    for url in filing_url:
        text_1 = extractorApi.get_section(url, "1", "text")
        text_1A = extractorApi.get_section(url, "1A", "text")
        text_1B = extractorApi.get_section(url, "1B", "text")
        text_2 = extractorApi.get_section(url, "2", "text")
        text_3 = extractorApi.get_section(url, "3", "text")
        text_4 = extractorApi.get_section(url, "4", "text")
        text_5 = extractorApi.get_section(url, "5", "text")
        text_6 = extractorApi.get_section(url, "6", "text")
        text_7 = extractorApi.get_section(url, "7", "text")
        text_7A = extractorApi.get_section(url, "7A", "text")
        text_8 = extractorApi.get_section(url, "8", "text")
        text_9 = extractorApi.get_section(url, "9", "text")
        text_9A = extractorApi.get_section(url, "9A", "text")
        text_9B = extractorApi.get_section(url, "9B", "text")
        text_10 = extractorApi.get_section(url, "10", "text")
        text_11 = extractorApi.get_section(url, "11", "text")
        text_12 = extractorApi.get_section(url, "12", "text")
        text_13 = extractorApi.get_section(url, "13", "text")
        text_14 = extractorApi.get_section(url, "14", "text")
        Business_1.append(text_1)
        RiskFactors_1A.append(text_1A) 
        UnresolvedStaffComments_1B.append(text_1B) 
        Properties_2.append(text_2) 
        LegalProceedings_3.append(text_3) 
        MineSafety_4.append(text_4) 
        MarketforEquityMatters_5.append(text_5) 
        SelectFinDatapreFeb21_6.append(text_6) 
        MgmtAnalysis_7.append(text_7) 
        MarketRisk_7A.append(text_7A) 
        FinStatements_8.append(text_8) 
        AccountingDiscrepancies_9.append(text_9) 
        Controls_9A.append(text_9A) 
        OtherInfo_9B.append(text_9B) 
        CorporateGovernance_10.append(text_10) 
        ExecComp_11.append(text_11) 
        BeneficialOwners_12.append(text_12) 
        DirectorIndependence_13.append(text_13) 
        AccountantFees_14.append(text_14)

    data = {"url": url, 
            "Business_1": Business_1,
            "RiskFactors_1A": RiskFactors_1A,
            "UnresolvedStaffComments_1B": UnresolvedStaffComments_1B,
            "Properties_2": Properties_2,
            "LegalProceedings_3": LegalProceedings_3,
            "MineSafety_4": MineSafety_4,
            "MarketforEquityMatters_5": MarketforEquityMatters_5,
            "SelectFinDatapreFeb21_6": SelectFinDatapreFeb21_6,
            "MgmtAnalysis_7": MgmtAnalysis_7,
            "MarketRisk_7A": MarketRisk_7A,
            "FinStatements_8": FinStatements_8,
            "AccountingDiscrepancies_9": AccountingDiscrepancies_9,
            "Controls_9A": Controls_9A,
            "OtherInfo_9B": OtherInfo_9B,
            "CorporateGovernance_10": CorporateGovernance_10,
            "ExecComp_11": ExecComp_11,
            "BeneficialOwners_12": BeneficialOwners_12,
            "DirectorIndependence_13": DirectorIndependence_13,
            "AccountantFees_14": AccountantFees_14}
    df = pd.DataFrame(data)
    return df
    

In [197]:
test_df = create_10k_textdf()

In [199]:
test_df.tail()

,url,Business_1,RiskFactors_1A,UnresolvedStaffComments_1B,Properties_2,LegalProceedings_3,MineSafety_4,MarketforEquityMatters_5,SelectFinDatapreFeb21_6,MgmtAnalysis_7,MarketRisk_7A,FinStatements_8,AccountingDiscrepancies_9,Controls_9A,OtherInfo_9B,CorporateGovernance_10,ExecComp_11,BeneficialOwners_12,DirectorIndependence_13,AccountantFees_14
140,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,ITEM 1: BUSINESS \n\nCorporate History \n\nWe...,ITEM 1A: RISK FACTORS \n\nOur operations and ...,ITEM 1B: UNRESOLVED STAFF COMMENTS \n\nNone. ...,ITEM 2: PROPERTIES \n\nOur corporate headquar...,ITEM 3. LEGAL PROCEEDINGS \n\nThree securitie...,ITEM 4: MINE SAFETY DISCLOSURES \n\nNot appli...,ITEM 5: MARKET FOR REGISTRANT&#8217;S COMMON ...,ITEM 6: SELECTED CONSOLIDATED FINANCIAL DATA ...,ITEM 7: MANAGEMENT&#8217;S DISCUSSION AND ANA...,ITEM 7A: QUANTITATIVE AND QUALITATIVE DISCLOS...,ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTAR...,ITEM 9: CHANGES IN AND DISAGREEMENTS WITH ACC...,ITEM 9A: &#32; CONTROLS &#32; AND PROCEDURES ...,ITEM 9B: OTHER INFORMATION \n\nNone. \n\nPART...,"ITEM 10. DIRECTORS, EXECUTIVE OFFICERS AND CO...",ITEM 11: EXECUTIVE COMPENSATION \n\n&#160; \n...,ITEM 12: SECURITY OWNERSHIP OF CERTAIN BENEFI...,ITEM 13: CERTAIN RELATIONSHIPS AND RELATED TR...,ITEM 14: PRINCIPAL ACCOUNTANT FEES AND SERVIC...
141,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined
142,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,ITEM 1. BUSINESS \n\nOverview \n\nUber Techno...,ITEM 1A. RISK FACTORS \n\nCertain factors may...,ITEM 1B. UNRESOLVED STAFF COMMENTS \n\nNot ap...,"ITEM 2. PROPERTIES \n\nAs of December 31, 202...",ITEM 3. LEGAL PROCEEDINGS \n\nWe are a party ...,ITEM 4. MINE SAFETY DISCLOSURES \n\nNot appli...,ITEM 5. MARKET FOR REGISTRANT&#8217;S COMMON ...,ITEM 6. [RESERVED] \n\n,ITEM 7. MANAGEMENT&#8217;S DISCUSSION AND ANA...,ITEM 7A. QUANTITATIVE AND QUALITATIVE DISCLOS...,ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTAR...,ITEM 9. CHANGES IN AND DISAGREEMENTS WITH ACC...,ITEM 9A. CONTROLS AND PROCEDURES \n\nEvaluati...,ITEM 9B. OTHER INFORMATION \n\nNot applicable...,"ITEM 10. DIRECTORS, EXECUTIVE OFFICERS AND CO...",ITEM 11. EXECUTIVE COMPENSATION \n\nThe infor...,ITEM 12. SECURITY OWNERSHIP OF CERTAIN BENEFI...,ITEM 13. CERTAIN RELATIONSHIPS AND RELATED TR...,ITEM 14. PRINCIPAL ACCOUNTING FEES AND SERVIC...
143,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,ITEM 1. BUSINESS \n\nOverview \n\nWe are a te...,ITEM 1A. RISK FACTORS \n\nCertain factors may...,ITEM 1B. UNRESOLVED STAFF COMMENTS \n\nNot ap...,"ITEM 2. PROPERTIES \n\nAs of December 31, 202...",ITEM 3. LEGAL PROCEEDINGS \n\nWe are a party ...,ITEM 4. MINE SAFETY DISCLOSURES \n\nNot appli...,ITEM 5. MARKET FOR REGISTRANT&#8217;S COMMON ...,ITEM 6. SELECTED FINANCIAL DATA \n\nThe follo...,ITEM 7. MANAGEMENT&#8217;S DISCUSSION AND ANA...,ITEM 7A. QUANTITATIVE AND QUALITATIVE DISCLOS...,ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTAR...,ITEM 9. CHANGES IN AND DISAGREEMENTS WITH ACC...,ITEM 9A. CONTROLS AND PROCEDURES \n\nEvaluati...,ITEM 9B. OTHER INFORMATION \n\nNot applicable...,"ITEM 10. DIRECTORS, EXECUTIVE OFFICERS AND CO...",ITEM 11. EXECUTIVE COMPENSATION \n\nThe infor...,ITEM 12. SECURITY OWNERSHIP OF CERTAIN BENEFI...,ITEM 13. CERTAIN RELATIONSHIPS AND RELATED TR...,ITEM 14. PRINCIPAL ACCOUNTING FEES AND SERVIC...
144,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,ITEM 1. BUSINESS \n\nOverview \n\nWe are a te...,ITEM 1A. RISK FACTORS \n\nCertain factors may...,ITEM 1B. UNRESOLVED STAFF COMMENTS \n\nNot ap...,"ITEM 2. PROPERTIES \n\nAs of December 31, 201...",ITEM 3. LEGAL PROCEEDINGS \n\nWe are a party ...,ITEM 4. MINE SAFETY DISCLOSURES \n\nNot appli...,ITEM 5. MARKET FOR REGISTRANT&#8217;S COMMON ...,ITEM 6. SELECTED FINANCIAL DATA \n\nThe follo...,ITEM 7. MANAGEMENT&#821

In [200]:
test_df.to_csv('../Notebooks/Data/Raw_Data/SEC_10k_Text.csv')

https://github.com/janlukasschroeder/nlp-cheat-sheet-python/blob/master/NLP-Cheat-Sheet.ipynb

In [ ]:
pd.read_csv(Notebooks/Data/Raw_Data/SEC_10k_Text.csv)

### Need to add Function to create 10-q text dataframe

### Randomness

In [177]:
no_10q_list = sec_section_df['sectionName_10q'].sort_values().to_list()
no_10q_list

['1-NA-10q',
 '10-NA-10q',
 '11-NA-10q',
 '12-NA-10q',
 '13-NA-10q',
 '14-NA-10q',
 '1B-NA-10q',
 '2-NA-10q',
 '6-NA-10q',
 '9-NA-10q',
 'part1item1-Fin Statements',
 'part1item2-Mgmt Analysis',
 'part1item3-Market Risk',
 'part1item4-Control',
 'part2item1-Legal Proceedings',
 'part2item1A-Risk Factors',
 'part2item2-Unreg Sales of Equity',
 'part2item3-Defaults on Securities',
 'part2item4-Mine Safety',
 'part2item5-Other Info',
 'part2item6-Exh']

In [72]:
# SEC Sections as a dictionary
    section_dict_10k = {
        '1':'1-Business', 
        '1A':'1A-Risk Factors', 
        '1B':'1B-Unresolved Staff Comments', 
        '2':'2-Properties', 
        '3':'3-Legal Proceedings',
        '4':'4-Mine Safety', 
        '5':'5-Market for Equity Matters',
        '6':'6-Select Fin Data pre-Feb21', 
        '7':'7-Mgmt Analysis', 
        '7A':'7A-Market Risk', 
        '8':'8-Fin Statements', 
        '9':'9-Accounting Discrepancies', 
        '9A':'9A-Controls', 
        '9B':'9B-Other Info', 
        '10':'10-Corporate Governance', 
        '11':'11-Exec Comp', 
        '12':'12-Beneficial Owners', 
        '13':'13-Director Independence', 
        '14':'14-Accountant Fees'
        }

IndentationError: unexpected indent (3282266493.py, line 2)

    1 - Business
    1A - Risk Factors
    1B - Unresolved Staff Comments
    2 - Properties
    3 - Legal Proceedings
    4 - Mine Safety Disclosures
    5 - Market for Registrant’s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities
    6 - Selected Financial Data (prior to February 2021)
    7 - Management’s Discussion and Analysis of Financial Condition and Results of Operations
    7A - Quantitative and Qualitative Disclosures about Market Risk
    8 - Financial Statements and Supplementary Data
    9 - Changes in and Disagreements with Accountants on Accounting and Financial Disclosure
    9A - Controls and Procedures
    9B - Other Information
    10 - Directors, Executive Officers and Corporate Governance
    11 - Executive Compensation
    12 - Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters
    13 - Certain Relationships and Related Transactions, and Director Independence
    14 - Principal Accountant Fees and Services

#### First efforts to get the Query API working

In [ ]:
df_uber = pd.json_normalize(filings, record_path =['filings'])
df_uber_trim = df_uber[['ticker', 'companyName', 'cik', 'accessionNo', 'formType', 'periodOfReport', 'linkToFilingDetails']]
df_uber_trim.tail(1)

,ticker,companyName,cik,accessionNo,formType,periodOfReport,linkToFilingDetails
29,BLK,BlackRock Inc.,1364742,0001193125-12-343728,10-Q,NaN,https://www.sec.gov/Archives/edgar/data/136474...


In [ ]:
from sec_api import QueryApi

queryApi = QueryApi(api_key="43d18e4ffedde4ca940e46ce210780daa0b7de17674abb369b5a8ef9fb3b69a1")

query = {
  "query": { "query_string": { 
      "query": "ticker:UBER AND filedAt:{2012-06-01 TO 2022-06-01} AND formType:\"10-Q\"" 
    } },
  "from": "0",
  "size": "50",
  "sort": [{ "filedAt": { "order": "desc" } }]
}

filings = queryApi.get_filings(query)

print(filings)

{'total': {'value': 10, 'relation': 'eq'}, 'query': {'from': 0, 'size': 10}, 'filings': [{'id': 'acab1c47e7c525bea4cd50c17e3a6635', 'accessionNo': '0001543151-22-000015', 'cik': '1543151', 'ticker': 'UBER', 'companyName': 'Uber Technologies, Inc', 'companyNameLong': 'Uber Technologies, Inc (Filer)', 'formType': '10-Q', 'description': 'Form 10-Q - Quarterly report [Sections 13 or 15(d)]', 'filedAt': '2022-05-05T16:07:20-04:00', 'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1543151/000154315122000015/0001543151-22-000015.txt', 'linkToHtml': 'https://www.sec.gov/Archives/edgar/data/1543151/0001543151-22-000015-index.htm', 'linkToXbrl': '', 'linkToFilingDetails': 'https://www.sec.gov/ix?doc=/Archives/edgar/data/1543151/000154315122000015/uber-20220331.htm', 'entities': [{'companyName': 'Uber Technologies, Inc (Filer)', 'cik': '1543151', 'irsNo': '452647441', 'stateOfIncorporation': 'DE', 'fiscalYearEnd': '1231', 'type': '10-Q', 'act': '34', 'fileNo': '001-38902', 'filmNo': '2289621